In [1]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval
import random


In [2]:
%run MODEL_LIST.ipynb            #Load the models

In [3]:
df = pd.read_csv("../data/ufc-master.csv")

In [4]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:

#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [6]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [7]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[:-200]
odds_train = odds_df[:-200]
label_train = label_df[:-200]

df_test = df[-200:]
odds_test = odds_df[-200:]
label_test = label_df[-200:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
3979
3979
3979


In [1]:
def get_ev(input_df, input_model, input_features, input_labels, odds_input, min_ev = 0):
    df_sel = input_df[input_features]
    df_sel = df_sel.dropna()
    df_sel = pd.get_dummies(df_sel)
    labels_sel = input_labels[input_labels.index.isin(df_sel.index)]
    odds_sel = odds_input[odds_input.index.isin(df_sel.index)] 
    best_score = custom_cv_eval(df_sel, input_model, labels_sel, odds_sel)
    return best_score

In [9]:
def tune_LogisticRegression(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. penalty ('l1' or 'l2')
    #2. tol (original_value, original_value * 1.2, original_value * 0.8, rand(0, 10)
    #3. random_state = 75
    #4. solver = 'newton-cg', 'lbfgs', 'sag', 'saga'    
    ###############################################################################################################
    print()
    print()
    print("Starting New Run for LogisticRegression")
    print()
    print()
    output_model = input_model
    best_score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
    print("Previous Best Score:", best_score)    

    
    penalty = ['l1', 'l2', 'none']
    solver = ['newton-cg', 'lbfgs', 'sag', 'saga']
    tol = [input_model.tol, input_model.tol * 1.2, input_model.tol * .8, random.random() * 10 ]
    for s in solver:
        score = -10000
        for p in penalty:
            for t in tol:
                if ((s == 'newton-cg') & (p == 'l1')) |\
                ((s == 'lbfgs') & (p == 'l1')) |\
                ((s == 'sag') & (p == 'l1')):

                    pass
                else:
                    test_model = LogisticRegression(solver = s, penalty = p, tol=t, random_state=75, max_iter=50000)
                    score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
                    if score > best_score:
                        best_score = score
                        output_model = test_model
                        
                        print()
                        print("NEW BEST SCORE")
                        print("solver:", s, 
                              "penalty:", p,
                              "tol:", t,
                              "Best Score:", best_score)        
                        print()
                        print()
                    else:
                        print("solver:", s, 
                              "penalty:", p,
                              "tol:", t,
                              "Score:", score)                                                       
    return(output_model)

In [10]:
def tune_DecisionTreeClassifier(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. criterion ('gini', 'entropy')
    #2. splitter ('random', 'best')
    #3. max_depth ('none', IF A NUMBER EXISTS +1, -1, random, else 2 RANDOM INTS 1->100)
    #4. min_samples_leaf(n-1, 0,  n+1)
    #5. max_leaf_nodes:('none', n+1, n-1, OR 4 random numbers)
    ###############################################################################################################
    print()
    print()
    print("Starting New Run for DecisionTree")
    print()
    print()
    output_model = input_model
    best_score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
    print("Previous Best Score:", best_score)    

    criterion = ['gini', 'entropy']
    splitter = ['random', 'best']
    if input_model.max_depth == None:
        max_depth = [None, random.randrange(100), random.randrange(100)]
    else:
        max_depth = [input_model.max_depth, input_model.max_depth - 1, input_model.max_depth + 1, random.randrange(100)]
        max_depth = [i for i in max_depth if i > 0]

    min_samples_leaf = [input_model.min_samples_leaf, input_model.min_samples_leaf - 1,
                         input_model.min_samples_leaf + 1, random.randrange(100)]
    min_samples_leaf = [i for i in min_samples_leaf if i > 0]    
    if input_model.max_leaf_nodes == None:
        max_leaf_nodes = [None, random.randrange(1000), random.randrange(1000)]
    else:
        max_leaf_nodes = [input_model.max_leaf_nodes, input_model.max_leaf_nodes - 1, 
                     input_model.max_leaf_nodes + 1, random.randrange(1000)]
        max_leaf_nodes = [i for i in max_leaf_nodes if i > 0]
    
    for l in max_leaf_nodes:
        for sam in min_samples_leaf:
            for m in max_depth:
                for c in criterion:
                    for s in splitter:
                        test_model = DecisionTreeClassifier(criterion = c, splitter = s, max_depth = m,
                                                            min_samples_leaf=sam, max_leaf_nodes = l, random_state=75)
                        score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
                        if score > best_score:
                            best_score = score
                            output_model = test_model
                            print()
                            print("NEW BEST SCORE")
                            
                            print("Criterion:", c, "splitter:", s, "max_depth:", m, 
                                  "min_samples_leaf:", sam, "max_leaf_nodes:", l, best_score)        
                            print()
                        else:
                            print("Criterion:", c, "splitter:", s, "max_depth:", m, 
                                  "min_samples_leaf:", sam, "max_leaf_nodes:", l, best_score)        
                            
                                        
    
    return output_model


In [11]:
def tune_RandomForestClassifier(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. criterion ('gini', 'entropy')
    #2. max_features ('auto', 'sqrt', 'log2')
    #3. max_depth ('none', IF A NUMBER EXISTS +2, -2, ELSE 2 RANDOM INTS 1->100)
    #4. min_samples_leaf(n-2, 0, n+2)
    #5. max_leaf_nodes:('none', n+2, n-2, OR 2 random numbers)
    #6. n_estimators: (n, n+2, n-2)
    ###############################################################################################################    
    print()
    print()
    print("Starting New Run for RandomForestClassifier")
    print()
    print()
    output_model = input_model
    best_score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
    print("Previous Best Score:", best_score)        
    #1. criterion ('gini', 'entropy')
    criterion = ['gini', 'entropy']
    #2. max_features ('auto', 'log2')
    max_features = ['auto', 'log2', None]
    #3. max_depth ('none', IF A NUMBER EXISTS +2, +4, -2, -4 ELSE 4 RANDOM INTS 1->100)
    if input_model.max_depth == None:
        max_depth = [None, random.randrange(100), random.randrange(100)]
    else:
        max_depth = [input_model.max_depth, input_model.max_depth - 2,   
                     input_model.max_depth + 2, random.randrange(100)]
        max_depth = [i for i in max_depth if i > 0]
    #4. min_samples_leaf(n-1, n-2, 0,  n+1, n+2)
    min_samples_leaf = [input_model.min_samples_leaf, input_model.min_samples_leaf - 2, 
                         input_model.min_samples_leaf + 2, random.randrange(100)]
    min_samples_leaf = [i for i in min_samples_leaf if i > 0]
    
    #5. max_leaf_nodes:('none', n+1, n+2, n-1, n-2, OR 4 random numbers)
    if input_model.max_leaf_nodes == None:
        max_leaf_nodes = [None, random.randrange(1000), random.randrange(1000)]
    else:
        max_leaf_nodes = [input_model.max_leaf_nodes, input_model.max_leaf_nodes - 2,  
                     input_model.max_leaf_nodes + 2, random.randrange(1000)]
        max_leaf_nodes = [i for i in max_leaf_nodes if i > 0]
    n_estimators = [input_model.n_estimators, input_model.n_estimators - 2,   
                 input_model.n_estimators + 2, random.randrange(200)]
    n_estimators = [i for i in n_estimators if i > 0]
    
    
    
    for n in n_estimators:
        for ml in max_leaf_nodes:
            for ms in min_samples_leaf:
                for md in max_depth:
                    for mf in max_features:
                        for c in criterion:
                            test_model = RandomForestClassifier(n_estimators = n, max_leaf_nodes = ml, 
                                                                min_samples_leaf = ms,
                                                                max_depth = md, criterion = c, 
                                                                max_features = mf, 
                                                                n_jobs = -1,
                                                                random_state=75)
                            score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
                            if score > best_score:
                                best_score = score
                                output_model = test_model
                                print()
                                print("NEW BEST SCORE")
                                print("Criterion:", c, "max_features:", mf, "max_depth:", md, "min_samples_leaf:", ms,
                                      "max_leaf_nodes:", ml, "n_estimators", n, best_score)        
                                print()
                                print()
                            else:
                                print("Criterion:", c, "max_features:", mf, "max_depth:", md, "min_samples_leaf:", ms,
                                      "max_leaf_nodes:", ml, "n_estimators", n, score)        
                            
    return output_model

In [35]:
def tune_GradientBoostingClassifier(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. criterion ('friedman_mse', 'mse', 'mae')
    #2. loss ('deviance', 'exponential')
    #3. n_estimators (n, n+1, n-1)
    #4. learning_rate (learning_rate, learning_rate *1.1, learning_rate*.9)
    #5. min_samples_leaf: (n, n-1, n+1)
    #6. max_depth: (n, n+1, n-1)
    #7. max_features: (None, 'auto', 'sqrt', 'log2')
    #8. max_leaf_nodes: (None, n+1, n-1, OR 2 random numbers)
    #9. tol (n, n*1.1, n*.9)
    ###############################################################################################################  
    print()
    print()
    print("Starting New Run")
    print()
    print()
    output_model = input_model
    best_score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
    print("Previous Best Score:", best_score)
    
    #1. criterion ('friedman_mse', 'mse', 'mae')
    criterion = ['friedman_mse']
    
    #2. loss ('deviance', 'exponential')
    loss = ['deviance']

    #3. n_estimators (n, n+1, n-1)
    n_estimators = [input_model.n_estimators, input_model.n_estimators - 1,  input_model.n_estimators + 1,
                    random.randrange(200)]
    n_estimators = [i for i in n_estimators if i > 0]    
    
    #4. learning_rate (learning_rate, learning_rate *1.1, learning_rate*.9)
    learning_rate = [input_model.learning_rate]
    
    #5. min_samples_leaf: (n, n-1, n+1)
    min_samples_leaf = [input_model.min_samples_leaf, input_model.min_samples_leaf - 1,
                         input_model.min_samples_leaf + 1]
    min_samples_leaf = [i for i in min_samples_leaf if i > 0]

    #6. max_depth: (n, n+1, n-1)
    if input_model.max_depth == None:
        max_depth = [None, random.randrange(100), random.randrange(100)]
    else:
        max_depth = [input_model.max_depth, input_model.max_depth - 1,  
                     input_model.max_depth + 1, random.randrange(100)]
        max_depth = [i for i in max_depth if i > 0]
        
    #7. max_features: (None, 'auto', 'sqrt', 'log2')
    max_features = ['sqrt', 'log2', None]

    #8. max_leaf_nodes: (None, n+1, n-1, OR 2 random numbers)
    if input_model.max_leaf_nodes == None:
        max_leaf_nodes = [None, random.randrange(1000), random.randrange(1000)]
    else:
        max_leaf_nodes = [input_model.max_leaf_nodes, input_model.max_leaf_nodes - 1, input_model.max_leaf_nodes + 1, 
                          random.randrange(1000)]
        max_leaf_nodes = [i for i in max_leaf_nodes if i > 0]

    #9. tol (n, n*1.1, n*.9)
    tol = [input_model.tol, input_model.tol * 1.2, input_model.tol * .8]
            
    print(len(tol) * len(max_leaf_nodes) * len(max_features) * len(max_depth) * len(min_samples_leaf) * len(learning_rate) * len(n_estimators) * len(loss) * len(criterion))    
        
        
    for t in tol:
        for ml in max_leaf_nodes:    
            for mf in max_features:
                for md in max_depth:
                    for ms in min_samples_leaf:
                        for lr in learning_rate:
                            for n in n_estimators:
                                for l in loss:
                                    for c in criterion:
                                        test_model = GradientBoostingClassifier(n_estimators = n, 
                                                                                learning_rate = lr,
                                                                                criterion = c,
                                                                                min_samples_leaf = ms,
                                                                                max_depth = md,
                                                                                loss = l, 
                                                                                max_features = mf,
                                                                                max_leaf_nodes = ml,
                                                                                tol = t,
                                                                                random_state=75)
                                        score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
                                        if score > best_score:
                                            best_score = score
                                            output_model = test_model
                                            print()
                                            print("NEW BEST SCORE")
                                            print("Criterion:", c,
                                                  "n_estimators:", n,
                                                  "Loss:", l,
                                                  "Learning Rate:", lr,
                                                  "Min Samples/Leaf:", ms,
                                                  "Max Depth:", md,
                                                  "Max Features:", mf,
                                                  "Max Leaf Nodes:", ml,
                                                  "tol:", t,
                                                  "Best Score:", best_score)        
                                            print()
                                            print()
                                        else:
                                            print("Criterion:", c,
                                                  "n_estimators:", n,                          
                                                  "Loss:", l, 
                                                  "Learning Rate:", lr,
                                                  "Min Samples/Leaf:", ms,
                                                  "Max Depth:", md,
                                                  "Max Features:", mf,
                                                  "Max Leaf Nodes:", ml,
                                                  "tol:", t,
                                                  "Score:", score)        

    
    return(output_model)

In [42]:
def tune_GaussianNB(input_model, input_features, input_df, input_labels, odds_input):
    ###############################################################################################################
    #Parameters we are going to fine-tune:
    #1. var_smoothing (1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6)
    ###############################################################################################################  
    print()
    print()
    print("Starting New Run for GaussianNB")
    print()
    print()
    output_model = input_model
    best_score = get_ev(input_df, input_model, input_features, input_labels, odds_input)
    print("Previous Best Score:", best_score)    
    
    var_smoothing = [1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6]
    
    for v in var_smoothing:
        test_model = GaussianNB(var_smoothing = v)
        score = get_ev(input_df, test_model, input_features, input_labels, odds_input)
        if score > best_score:
            best_score = score
            output_model = test_model
            print()
            print("NEW BEST SCORE")
            print("var_smoothing:", v, 
                  "Best Score:", best_score)        
            print()
            print()
        else:
            print("var_smoothing:", v, 
                  "Score:", score)        
        
    
    return output_model

In [43]:
def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    best_model = input_model
    keep_going = True
    
    if isinstance(input_model, LogisticRegression):
        while(keep_going):
            pos_model = (tune_LogisticRegression(best_model, input_features, input_df, input_labels, odds_input))
            
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model
                
    elif isinstance(input_model, DecisionTreeClassifier):
        while(keep_going):
            pos_model = (tune_DecisionTreeClassifier(best_model, input_features, input_df, input_labels, odds_input))
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model            
                
    elif isinstance(input_model, RandomForestClassifier):
        while(keep_going):
            pos_model = (tune_RandomForestClassifier(best_model, input_features, input_df, input_labels, odds_input))
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model    
                                
    elif isinstance(input_model, GradientBoostingClassifier):
        print("HI")
        while(keep_going):
            pos_model = (tune_GradientBoostingClassifier(best_model, input_features, input_df, input_labels, odds_input))
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model                    
                
    elif isinstance(input_model, GaussianNB):
        while(keep_going):
            pos_model = (tune_GaussianNB(best_model, input_features, input_df, input_labels, odds_input))
            if str(pos_model) == str(best_model):  #Direct comparisons don't seem to work....
                keep_going = False
                output_model = best_model
            else:
                best_model = pos_model                    
                
                
    else:
        output_model = input_model
    return(output_model)                

In [44]:
return_model = tune_hyperparameters(model_5, features_5, df_train, label_train, odds_train)



Starting New Run for GaussianNB


Previous Best Score: 26.548529006243612
var_smoothing: 1e-12 Score: 22.16188462837639
var_smoothing: 1e-11 Score: 22.395383614225665
var_smoothing: 1e-10 Score: 21.98539764209508
var_smoothing: 1e-09 Score: 26.548529006243612
var_smoothing: 1e-08 Score: 24.371462490996667
var_smoothing: 1e-07 Score: 12.395095263188992
var_smoothing: 1e-06 Score: 11.01563661452688


In [45]:
print(return_model)

GaussianNB(priors=None, var_smoothing=1e-09)


In [37]:
return_model = tune_hyperparameters(model_4, features_4, df_train, label_train, odds_train)

HI


Starting New Run


Previous Best Score: 25.57415008340652
864
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: sqrt Max Leaf Nodes: None tol: 0.0001 Score: 2.5180308054039195
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: sqrt Max Leaf Nodes: None tol: 0.0001 Score: 2.73854887265299
Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: sqrt Max Leaf Nodes: None tol: 0.0001 Score: 1.4949550195564623
Criterion: friedman_mse n_estimators: 57 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: sqrt Max Leaf Nodes: None tol: 0.0001 Score: -14.974518374364994
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: sqrt Max Leaf Nodes: None tol: 0.0001 Score: 1.5253980465492756
C

Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 2 Max Features: log2 Max Leaf Nodes: None tol: 0.0001 Score: -17.28383195840437
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 2 Max Features: log2 Max Leaf Nodes: None tol: 0.0001 Score: -17.210233130928103
Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 2 Max Features: log2 Max Leaf Nodes: None tol: 0.0001 Score: -15.006068175795171
Criterion: friedman_mse n_estimators: 57 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 2 Max Features: log2 Max Leaf Nodes: None tol: 0.0001 Score: -20.049805207685488
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: log2 Max Leaf Nodes: None tol: 0.0001 Score: 6.745979602143243
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning 

Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 28 Max Features: None Max Leaf Nodes: None tol: 0.0001 Score: 7.160910091307592
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 28 Max Features: None Max Leaf Nodes: None tol: 0.0001 Score: 7.350796223879892
Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 28 Max Features: None Max Leaf Nodes: None tol: 0.0001 Score: 7.454645462220889
Criterion: friedman_mse n_estimators: 57 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 28 Max Features: None Max Leaf Nodes: None tol: 0.0001 Score: 9.899202371243888
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 28 Max Features: None Max Leaf Nodes: None tol: 0.0001 Score: 2.856308737112436
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Ra

Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: log2 Max Leaf Nodes: 243 tol: 0.0001 Score: -9.209149943166196
Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: log2 Max Leaf Nodes: 243 tol: 0.0001 Score: -7.642419436809884
Criterion: friedman_mse n_estimators: 57 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: log2 Max Leaf Nodes: 243 tol: 0.0001 Score: -17.41909015487126
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 2 Max Features: log2 Max Leaf Nodes: 243 tol: 0.0001 Score: -15.470741213514568
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 2 Max Features: log2 Max Leaf Nodes: 243 tol: 0.0001 Score: -15.943763162385618
Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate:

Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: None Max Leaf Nodes: 243 tol: 0.0001 Score: 16.578844850062797
Criterion: friedman_mse n_estimators: 57 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: None Max Leaf Nodes: 243 tol: 0.0001 Score: 14.56682987342542
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 4 Max Features: None Max Leaf Nodes: 243 tol: 0.0001 Score: 11.355947498208785
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 4 Max Features: None Max Leaf Nodes: 243 tol: 0.0001 Score: 12.184194215402506
Criterion: friedman_mse n_estimators: 101 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 4 Max Features: None Max Leaf Nodes: 243 tol: 0.0001 Score: 12.331801822951347
Criterion: friedman_mse n_estimators: 57 Loss: deviance Learning Rate: 0.

...

Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 2 Max Features: None Max Leaf Nodes: None tol: 8e-05 Score: 21.298527862403844
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 2 Max Features: None Max Leaf Nodes: None tol: 8e-05 Score: 22.13402678857917
Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 2 Max Features: None Max Leaf Nodes: None tol: 8e-05 Score: 9.16237221643604
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 2 Max Features: None Max Leaf Nodes: None tol: 8e-05 Score: 20.037743602246188
Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 2 Max Features: None Max Leaf Nodes: None tol: 8e-05 Score: 19.306845945542285
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 M

Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 226 tol: 8e-05 Score: -2.5824998203220018
Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 226 tol: 8e-05 Score: -19.163417203456312
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 84 Max Features: sqrt Max Leaf Nodes: 226 tol: 8e-05 Score: 4.193006703347452
Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 84 Max Features: sqrt Max Leaf Nodes: 226 tol: 8e-05 Score: 4.027255299479927
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 84 Max Features: sqrt Max Leaf Nodes: 226 tol: 8e-05 Score: 4.7389016750118795
Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 

Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: None Max Leaf Nodes: 226 tol: 8e-05 Score: 18.61908711648178
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: None Max Leaf Nodes: 226 tol: 8e-05 Score: 12.565563453828322
Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: None Max Leaf Nodes: 226 tol: 8e-05 Score: 14.566889339997898
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: None Max Leaf Nodes: 226 tol: 8e-05 Score: 14.355607500802929
Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 3 Max Features: None Max Leaf Nodes: 226 tol: 8e-05 Score: 15.0955018834312
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Sam

Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 379 tol: 8e-05 Score: 1.6066784266966039
Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 379 tol: 8e-05 Score: 1.9420046599509142
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 379 tol: 8e-05 Score: 3.060086812783285
Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 379 tol: 8e-05 Score: -21.16395040553067
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 4 Max Features: sqrt Max Leaf Nodes: 379 tol: 8e-05 Score: -2.523665190243042
Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min S

Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 84 Max Features: log2 Max Leaf Nodes: 379 tol: 8e-05 Score: -3.5868285152858026
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 84 Max Features: log2 Max Leaf Nodes: 379 tol: 8e-05 Score: -3.4542439876202025
Criterion: friedman_mse n_estimators: 36 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 2 Max Depth: 84 Max Features: log2 Max Leaf Nodes: 379 tol: 8e-05 Score: -0.6175489685485562
Criterion: friedman_mse n_estimators: 99 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: None Max Leaf Nodes: 379 tol: 8e-05 Score: 20.204110836726706
Criterion: friedman_mse n_estimators: 98 Loss: deviance Learning Rate: 0.1 Min Samples/Leaf: 1 Max Depth: 3 Max Features: None Max Leaf Nodes: 379 tol: 8e-05 Score: 19.640774223685973
Criterion: friedman_mse n_estimators: 100 Loss: deviance Learning Rate: 0

In [38]:
print(return_model)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=99,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=75, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [51]:
return_model = tune_hyperparameters(model_3, features_3, df_train, label_train, odds_train)



Starting New Run for RandomForestClassifier


Previous Best Score: 33.11951036936883
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 33.11951036936883
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 25.477004601917486
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 7.140418113887313
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 -1.030691962146741
Criterion: gini max_features: None max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 17.57120304428968
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 14.877134106638016
Criterion: gini max_features: auto max_depth: 85 min_samples_leaf: 1 max_leaf_nodes: None n_estimators 100 -6.534557948548727
Criterion: en

Criterion: entropy max_features: log2 max_depth: 85 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 -15.319048698791343
Criterion: gini max_features: None max_depth: 85 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 11.327589229513503
Criterion: entropy max_features: None max_depth: 85 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 10.292991613671262
Criterion: gini max_features: auto max_depth: 43 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 -10.67609267117462
Criterion: entropy max_features: auto max_depth: 43 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 -12.395633175943718
Criterion: gini max_features: log2 max_depth: 43 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 -16.65027355042083
Criterion: entropy max_features: log2 max_depth: 43 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 -16.843339579981894
Criterion: gini max_features: None max_depth: 43 min_samples_leaf: 1 max_leaf_nodes: 631 n_estimators 100 7.72

Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -17.49157779252079
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -17.52523035246415
Criterion: gini max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -13.829466916715102
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -14.282396995494384
Criterion: gini max_features: auto max_depth: 85 min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -15.2129676976441
Criterion: entropy max_features: auto max_depth: 85 min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -15.973645153513846
Criterion: gini max_features: log2 max_depth: 85 min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -17.49157779252079
Criterion: entropy max_features: log2 max_depth: 85 min_samples_leaf: 3 max_leaf_nodes: 4 n_estimators 100 -17.525230352

Criterion: entropy max_features: auto max_depth: 43 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 -15.880416457330039
Criterion: gini max_features: log2 max_depth: 43 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 -16.893583205989323
Criterion: entropy max_features: log2 max_depth: 43 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 -16.877904648651754
Criterion: gini max_features: None max_depth: 43 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 -4.065216358418669
Criterion: entropy max_features: None max_depth: 43 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 98 -7.782808046918999
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 54 max_leaf_nodes: None n_estimators 98 -15.940280429198232
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 54 max_leaf_nodes: None n_estimators 98 -15.940280429198232
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 54 max_leaf_nodes: None n_estimat

Criterion: gini max_features: auto max_depth: 85 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -15.940280429198232
Criterion: entropy max_features: auto max_depth: 85 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -15.940280429198232
Criterion: gini max_features: log2 max_depth: 85 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -17.27944353242299
Criterion: entropy max_features: log2 max_depth: 85 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -17.27944353242299
Criterion: gini max_features: None max_depth: 85 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -21.548779755635394
Criterion: entropy max_features: None max_depth: 85 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -20.102170178603185
Criterion: gini max_features: auto max_depth: 43 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -15.940280429198232
Criterion: entropy max_features: auto max_depth: 43 min_samples_leaf: 54 max_leaf_nodes: 631 n_estimators 98 -1

...

Criterion: entropy max_features: None max_depth: 31 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -6.46938725686299
Criterion: gini max_features: auto max_depth: 15 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -16.712233079741072
Criterion: entropy max_features: auto max_depth: 15 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -17.799557755164777
Criterion: gini max_features: log2 max_depth: 15 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -17.811824997997732
Criterion: entropy max_features: log2 max_depth: 15 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -16.32146656452493
Criterion: gini max_features: None max_depth: 15 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -9.585703099369319
Criterion: entropy max_features: None max_depth: 15 min_samples_leaf: 1 max_leaf_nodes: 925 n_estimators 100 -12.990846075307765
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 3 max_leaf_nodes: 925 n_estimators 100 -1

Criterion: gini max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -2.821771585383803
Criterion: entropy max_features: None max_depth: None min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -6.6165270230231465
Criterion: gini max_features: auto max_depth: 31 min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -15.80981128291409
Criterion: entropy max_features: auto max_depth: 31 min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -14.956130344878298
Criterion: gini max_features: log2 max_depth: 31 min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -15.820578285018014
Criterion: entropy max_features: log2 max_depth: 31 min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -16.148141701199837
Criterion: gini max_features: None max_depth: 31 min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100 -3.944106670475044
Criterion: entropy max_features: None max_depth: 31 min_samples_leaf: 3 max_leaf_nodes: 630 n_estimators 100

Criterion: gini max_features: log2 max_depth: 15 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 163 -17.130286624468205
Criterion: entropy max_features: log2 max_depth: 15 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 163 -17.147444927395735
Criterion: gini max_features: None max_depth: 15 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 163 -7.938589288720715
Criterion: entropy max_features: None max_depth: 15 min_samples_leaf: 3 max_leaf_nodes: None n_estimators 163 -7.5746968492496745
Criterion: gini max_features: auto max_depth: None min_samples_leaf: 10 max_leaf_nodes: None n_estimators 163 -15.907857575866327
Criterion: entropy max_features: auto max_depth: None min_samples_leaf: 10 max_leaf_nodes: None n_estimators 163 -15.548692061097455
Criterion: gini max_features: log2 max_depth: None min_samples_leaf: 10 max_leaf_nodes: None n_estimators 163 -17.3646538254754
Criterion: entropy max_features: log2 max_depth: None min_samples_leaf: 10 max_leaf_nodes: None 

Criterion: gini max_features: auto max_depth: 31 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -16.98582471936975
Criterion: entropy max_features: auto max_depth: 31 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -16.626659204600877
Criterion: gini max_features: log2 max_depth: 31 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -17.414817069546274
Criterion: entropy max_features: log2 max_depth: 31 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -17.414817069546274
Criterion: gini max_features: None max_depth: 31 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -10.387146672642482
Criterion: entropy max_features: None max_depth: 31 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -11.380786476196384
Criterion: gini max_features: auto max_depth: 15 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimators 163 -16.98582471936975
Criterion: entropy max_features: auto max_depth: 15 min_samples_leaf: 10 max_leaf_nodes: 925 n_estimator

Criterion: gini max_features: None max_depth: 15 min_samples_leaf: 10 max_leaf_nodes: 630 n_estimators 163 -6.208432014069098
Criterion: entropy max_features: None max_depth: 15 min_samples_leaf: 10 max_leaf_nodes: 630 n_estimators 163 -10.643110794920823


In [52]:
print(return_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=98, n_jobs=-1,
                       oob_score=False, random_state=75, verbose=0,
                       warm_start=False)


In [34]:
tune_hyperparameters(model_2, features_2, df_train, label_train, odds_train)



Starting New Run for DecisionTree


Previous Best Score: 28.731138996088998
Criterion: gini splitter: random max_depth: None min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: gini splitter: best max_depth: None min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: entropy splitter: random max_depth: None min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: entropy splitter: best max_depth: None min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: gini splitter: random max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: gini splitter: best max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: entropy splitter: random max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: entropy splitter: best max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: None 28.731138996088998
Criterion: gini splitter: random max_depth: 55 min_sam

Criterion: gini splitter: random max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: gini splitter: best max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: entropy splitter: random max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: entropy splitter: best max_depth: 35 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: gini splitter: random max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: gini splitter: best max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: entropy splitter: random max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: entropy splitter: best max_depth: 55 min_samples_leaf: 1 max_leaf_nodes: 327 28.731138996088998
Criterion: gini splitter: random max_depth: None min_samples_leaf: 2 max_leaf_nodes: 327 28.731138996088998
Criterion: gini splitter: best max_depth

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')

In [31]:
tune_hyperparameters(model_1, features_1, df_train, label_train, odds_train)



Starting New Run for LogisticRegression


Previous Best Score: 31.158123326360922
solver: newton-cg penalty: l2 tol: 0.0001 Score: 27.91023675721005
solver: newton-cg penalty: l2 tol: 0.00012 Score: 27.91023675721005
solver: newton-cg penalty: l2 tol: 8e-05 Score: 27.91023675721005
solver: newton-cg penalty: l2 tol: 0.004596721758762801 Score: 27.91023675721005
solver: newton-cg penalty: none tol: 0.0001 Score: 28.40894614174509
solver: newton-cg penalty: none tol: 0.00012 Score: 28.40894614174509
solver: newton-cg penalty: none tol: 8e-05 Score: 28.40894614174509
solver: newton-cg penalty: none tol: 0.004596721758762801 Score: 28.40894614174509
solver: lbfgs penalty: l2 tol: 0.0001 Score: 31.158123326360922
solver: lbfgs penalty: l2 tol: 0.00012 Score: 31.158123326360922
solver: lbfgs penalty: l2 tol: 8e-05 Score: 31.158123326360922
solver: lbfgs penalty: l2 tol: 0.004596721758762801 Score: 31.158123326360922

NEW BEST SCORE
solver: lbfgs penalty: none tol: 0.0001 Best Score: 31.471

KeyboardInterrupt: 